In [21]:
import math
import time
import numpy as np
import pinocchio as pin
import example_robot_data as robex
from pinocchio.visualize import MeshcatVisualizer as Visualizer
import ilqr
import sys

In [2]:
robot = robex.load('ur5')

In [3]:
# Initialize the viewer.
try:
    viz = Visualizer(robot.model, robot.collision_model, robot.visual_model)
    viz.initViewer(loadModel=True)
except ImportError as err:
    print(
        "Error while initializing the viewer. "
        "It seems you should install gepetto-viewer"
    )
    print(err)
    sys.exit(0)

try:
    viz.loadViewerModel("pinocchio")
except AttributeError as err:
    print(
        "Error while loading the viewer model. "
        "It seems you should start gepetto-viewer"
    )
    print(err)
    sys.exit(0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7035/static/


In [27]:
q0 = np.random.rand(robot.model.nq) * 2 * np.pi - np.pi
viz.display(q0)
robot.model.gravity = pin.Motion.Zero() # disable gravity

In [43]:
dt = 0.01
T = 5
N = math.floor(T / dt)

state_dim = robot.model.nq + robot.model.nv
control_dim = robot.model.nq

Q = np.zeros((state_dim, state_dim))  # state cost
Qf = np.eye(state_dim)  # final state cost
R = 1e-3 * np.eye(control_dim)  # control cost (minimize the energy)

s = ilqr.ILQRSolver(state_dim, control_dim, Q, Qf, R)
def dynamics(x, u):
    q, v = x[:robot.model.nq], x[robot.model.nq:]

    # apply the ABA algorithm to compute the acceleration
    a = pin.aba(robot.model, robot.data, q, v, u)

    # integrate the acceleration to get the new state
    v += a * dt
    q = pin.integrate(robot.model, q, v * dt)

    return np.concat((q, v))


# target = np.concatenate((robot.q0, np.zeros(robot.model.nv)))   # stay where you are
target_q = np.random.rand(robot.model.nq) * 2 * np.pi - np.pi # random position
target = np.concatenate((target_q, np.zeros(robot.model.nv)))
controls = s.solve(np.concatenate((q0, np.zeros(robot.nv))), target, dynamics, time_steps=N, max_iterations=10, gradient_clip=10)

In [44]:
viz.display(q0)

In [45]:
q, v = q0, np.zeros(robot.model.nv)

t = 0.0
for k in range(N):
    tic = time.time()

    tau_control = controls[k]
    x = dynamics(np.concat((q, v)), tau_control)
    q, v = x[:robot.model.nq], x[robot.model.nq:]

    viz.display(q)

    toc = time.time()
    ellapsed = toc - tic
    dt_sleep = max(0, dt - (ellapsed))

    time.sleep(dt_sleep)
    t += dt